In [1]:
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import StandardScaler as zscore # zscore
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso #LRlasso
from joblib import dump, load #to save models in files
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import re
import json
from datetime import datetime
import os

In [2]:
with open("/home/t44p/PW_rawdata/results/newfeat_rfr/lcms/lcms_dict_nXcv.json", 'r') as json_file:
    lcms_prefix = json.load(json_file)
with open("/home/t44p/PW_rawdata/results/newfeat_rfr/gcms/gcms_dict_nXcv.json", 'r') as json_file:
    gcms_prefix = json.load(json_file)

In [3]:
gc = list(gcms_prefix.keys())
lc = list(lcms_prefix.keys())


In [4]:
print(lc)

['alpha_L_Fucose_1_Phosphate', 'Trehalose', 'Trehalose_phenolic_acid_', 'Maltotriose_', '_366_9939681_1_170199352', '_R_2_Phenylglycin', 'Oxidized_glutathione_', '_312_0943959_2_614228811', 'Guanosine_', 'gamma_Glutamylvaline_', '_512_1445532_3_132648409', 'Catechol_', '_303_0720727_3_335904942', 'Quinic_acid_derivative_3_56', '_374_1568694_3_674168475', 'Phaseoloidin', '_496_1502126_3_792994891', 'Vanilloloside', 'CGA_hexose_', 'Phenolic_glycoside_', 'gamma_Glutamylisoleucine_', 'L_gamma_glutamyl_L_isoleucine', '_658_1575839_4_528042834', 'Phenylalanylglutamic_acid_', '_5_Chlorogenic_acid', '_523_1663837_5_238225973', '_487_2029578_5_338775921', 'Quinic_acid_161_05_', 'Quinic_acid_derivative_with_he', 'Chlorogenic_acid_', 'Coumaroyl_quinic_acid_', '_351_1294837_5_809380415', 'Quercetin_glucose_glucose_rham', '_427_1824019_5_95992104', 'Quercetin_glc_glc_rha', '_567_2828405_6_133036019', 'Quercetin_3_7_di_glucoside_', '_441_1978503_6_218123415', '_4_O_p_Coumaroylquinic_acid_', 'Quercet

In [5]:
lasso_df = pd.read_csv("/home/t44p/PW_rawdata/results/newfeat_lasso/newfeat_lasso_performance.csv", sep="\t")
lasso_df = lasso_df[lasso_df['predictive_power'] == 'good'].copy()
lasso_good = list(lasso_df['target'])


In [6]:
rfr_df = pd.read_csv("/home/t44p/PW_rawdata/results/newfeat_rfr/newfeat_rfr_performance.csv", sep="\t")
rfr_df = rfr_df[rfr_df['predictive_power'] == 'good'].copy()
rfr_good = list(rfr_df['target'])


In [7]:
shared = set(rfr_good).intersection(set(lasso_good)) # shared by both, good predictors independent of approach
uniq_lasso = set(lasso_good).difference(set(rfr_good)) # uniq to lasso
uniq_rfr = set(rfr_good).difference(set(lasso_good)) # uniq to rfr

In [10]:
print(len(uniq_rfr))
print(len(uniq_lasso))
print(len(shared))


10
4
38


In [29]:
# Table 3. Good performing predictors shared by LASSO and Random Forest Regressor
for e in shared:
    # print(e)
    e = e.replace("_nXcv", "")
    if e in lc: 
        print(f"{e},LC-MS")
    elif e in gc:
        print(f"{e},GC-MS")

    


Isorhamnetin_3_O_glucoside_L,LC-MS
threonic_acid_292_220_rt7_49,GC-MS
Quercetin_glucose,LC-MS
p_coumaroyl_di_glucoside_L,LC-MS
pyroglutamic_acid_156_258_rt8_,GC-MS
Saponin_10_183,LC-MS
Quercetin_3_glucoside_3,LC-MS
alanine_116_218_rt3_38,GC-MS
Inosine_L,LC-MS
raffinose_437_451_rt16_91,GC-MS
CGA_hexose_,LC-MS
Kaempferol_3_O_rhamnoside_7_O_,LC-MS
_Rutin,LC-MS
erythronic_acid_292_rt7_24,GC-MS
glutamic_acid_246_363_rt8_31,GC-MS
Phenylalanylglutamic_acid_,LC-MS
Quercetin_7_4_O_diglucoside_,LC-MS
Maltotriose_,LC-MS
_312_0943959_2_614228811,LC-MS
Cellobiose_361_204_or_maltose_,GC-MS
Nicotinamide_adenine_dinucleot,LC-MS
Kaempferol_7_O_Glucoside_1_L,LC-MS
Quercitin_3_7_diglucoside_6_rh,LC-MS
glyceric_acid_292_189_rt5_63,GC-MS
aspartic_acid_232_218_rt7_48,GC-MS
Feruloyl_di_glucoside_L,LC-MS
Quercitin_3_O_Glucoside_2_L,LC-MS
naringenin_7_O_glucoside_1_L,LC-MS
_4_O_p_Coumaroylquinic_acid_,LC-MS
Caffeoylshikimic_acid_L,LC-MS
_1_O_Feruloyl_glucose_L,LC-MS
Phenylalanyaspartatic_acid_L,LC-MS
Oxidized_

In [33]:
# Table4: Good performing predictors uniq to lasso

for e in uniq_lasso:
    # print(e)
    e = e.replace("_nXcv", "")
    if e in lc: 
        print(f"{e},LC-MS")
    elif e in gc:
        print(f"{e},GC-MS")


Lactobionic_acid_L,LC-MS
glucose_1_phosphate_217_rt9_16,GC-MS
_658_1575839_4_528042834,LC-MS
L_gamma_glutamyl_L_isoleucine,LC-MS


In [31]:
# Table54: Good performing predictors uniq to rfr

for e in uniq_rfr:
    # print(e)
    e = e.replace("_nXcv", "")
    if e in lc: 
        print(f"{e},LC-MS")
    elif e in gc:
        print(f"{e},GC-MS")


Fructose_1_6_biosphasphate_L,LC-MS
_487_2029578_5_338775921,LC-MS
Cellobiose_361_204_rt14_40,GC-MS
_6_phospho_gluconate,GC-MS
_641_171822_6_885700966,LC-MS
Isorhamnetin_3_O_rutinoside,LC-MS
Kaempferol_glucose_xylose_L,LC-MS
Conduritol_B_Epoxide_L,LC-MS
Caffeic_acid_glucoside_L,LC-MS
Quinic_acid_161_05_,LC-MS
